In [31]:
import spatialdata
import sopa.segmentation
import sopa.io

## Create a `SpatialData` object

For this tutorial, we use a generated dataset. You can expect a total runtime of a few minutes.

To load your own data, see the commented lines below, or read the [`sopa.io` API](../../api/io).

In [14]:
# The line below creates a toy dataset for this tutorial
# To load your own data, such as MERSCOPE data, you can do `sdata = sopa.io.merscope("/path/to/region_0")`
# For more details, see https://gustaveroussy.github.io/sopa/api/io/
sdata = sopa.io.uniform()

sdata

Before starting, we create the variables below that denotes the names of the image and transcripts that we want to use, as displayed in the `SpatialData` object above:

In [15]:
image_key = "image"
points_key = "transcripts" # (ignore this for multiplex imaging)
gene_column = "genes" # (optional) column of sdata[points_key] containing the gene names

## Segmentation

### Option 1: Cellpose

First, we generate the bounding boxes of the patches on which Cellpose will be run. Here, the patches have a width and height of 1500 pixels and an overlap of 50 pixels. We advise bigger sizes for real datasets (see our default parameters in one of our [config files](https://github.com/gustaveroussy/sopa/tree/master/workflow/config)). On the toy dataset, this will generate **4** patches.

In [16]:
patches = sopa.segmentation.Patches2D(sdata, image_key, patch_width=1200, patch_overlap=50)
patches.write();

The following channels are available for segmentation. Choose one or two channels used by Cellpose.

In [17]:
from sopa._sdata import get_spatial_image

print(get_spatial_image(sdata, image_key).c.values)

Then, we initialize the Cellpose model. Here, we run segmentation using DAPI only, and we set the cell diameter to be about `35` pixels:

In [18]:
channels = ["DAPI"]

method = sopa.segmentation.methods.cellpose_patch(diameter=35, channels=channels, flow_threshold=2, cellprob_threshold=-6)
segmentation = sopa.segmentation.StainingSegmentation(sdata, method, channels, min_area=2500)

# The cellpose boundaries will be temporary saved here. You can choose a different path
cellpose_temp_dir = "tuto.zarr/.sopa_cache/cellpose"

#### Sequential running

If desired, you can run Cellpose sequentially, as in the lines below, but this is slower than the "Parallel running" section below.

In [ ]:
segmentation.write_patches_cells(cellpose_temp_dir)

#### Parallel running

Here, we show how to run Cellpose in parallel for all the patches. It's up to you to choose the way to parallelize it: for instance, if you have a Slurm cluster, you can run one job per patch.

Below, we run segmentation on each patch, and save the results in a temporary directory (here, `tuto.zarr/.sopa_cache`).
On this example, we performed a "for-loop", so you **should** paralellize this yourself using multiple jobs or multi-threading (see [this discussion](https://github.com/gustaveroussy/sopa/discussions/36) for more details). Note that you can also use our [Snakemake pipeline](https://gustaveroussy.github.io/sopa/tutorials/snakemake/) that will handle the parallelization for you.

In [7]:
# parallelize this for loop yourself (or use the Snakemake pipeline)
for patch_index in range(len(sdata['sopa_patches'])):
    segmentation.write_patch_cells(cellpose_temp_dir, patch_index)

#### Resolving conflicts

At this stage, you executed 4 times Cellpose (once per patch). Now, we need to resolve the conflict, i.e. where boundaries are overlapping due to segmentation on multiple patches:

In [20]:
cells = sopa.segmentation.StainingSegmentation.read_patches_cells(cellpose_temp_dir)
cells = sopa.segmentation.shapes.solve_conflicts(cells)

shapes_key = "cellpose_boundaries" # name of the key given to the cells in sdata.shapes

sopa.segmentation.StainingSegmentation.add_shapes(sdata, cells, image_key, shapes_key)

### Option 2: Baysor

In [4]:
shapes_key = "baysor_boundaries" # the name that we will give to the baysor "shapes"

Baysor needs a config to be executed. You can find official config examples [here](https://github.com/kharchenkolab/Baysor/tree/master/configs).

You can also reuse the Baysor parameter we have defined for each machine, as in our [Snakemake config files](https://github.com/gustaveroussy/sopa/tree/master/workflow/config). Note that, our Snakemake config is a `.yaml` file, but the Baysor config should still be a `.toml` file.

For this tutorial, we will use the config below. Instead of a dictionnary, you can also have a `.toml` file and provide `config_path` to the `patchify_transcripts` function.

In [5]:
config = {
    "data": {
        "force_2d": True,
        "min_molecules_per_cell": 10,
        "x": "x",
        "y": "y",
        "z": "z",
        "gene": "genes",
        "min_molecules_per_gene": 0,
        "min_molecules_per_segment": 3,
        "confidence_nn_id": 6
    },
    "segmentation": {
        "scale": 30,  # Important parameter: typical cell diameter, in microns (see our configs)
        "scale_std": "25%",
        "prior_segmentation_confidence": 0,
        "estimate_scale_from_centers": False,
        "n_clusters": 4,
        "iters": 500,
        "n_cells_init": 0,
        "nuclei_genes": "",
        "cyto_genes": "",
        "new_component_weight": 0.2,
        "new_component_fraction": 0.3
    }
}

Then, we generate the bounding boxes of the patches on which Baysor will be run. Here, the patches have a width and height of 3000 microns and an overlap of 50 microns. We advise bigger sizes for real datasets (see our default parameters in one of our [config files](https://github.com/gustaveroussy/sopa/tree/master/workflow/config)). On the toy dataset, this will generate **1** patch.

In [6]:
# The cellpose boundaries will be temporary saved here. You can choose a different path
baysor_temp_dir = "tuto.zarr/.sopa_cache/baysor"

patches = sopa.segmentation.Patches2D(sdata, points_key, patch_width=3000, patch_overlap=50)
valid_indices = patches.patchify_transcripts(baysor_temp_dir, config=config)

Now, we can run Baysor on each individual patch. It's up to you to choose the way to parallelize it: for instance, if you have a Slurm cluster, you can run one job per patch.

Below, we run segmentation on each patch, and save the results in a temporary directory (here, `tuto.zarr/.sopa_cache/baysor`).
On this example, we performed a "for-loop", so you **should** paralellize this yourself using multiple jobs or multi-threading (see [this discussion](https://github.com/gustaveroussy/sopa/discussions/36) for more details). Note that you can also use our [Snakemake pipeline](https://gustaveroussy.github.io/sopa/tutorials/snakemake/) that will handle the parallelization for you.

> NB: depending on you Baysor installation, you may need to update the `baysor_executable_path` variable to locate the Baysor binary executable

In [7]:
import subprocess

baysor_executable_path = "~/.julia/bin/baysor"

In [ ]:
for patch_index in valid_indices:
    command = f"""
    cd {baysor_temp_dir}/{patch_index}
    {baysor_executable_path} run --save-polygons GeoJSON -c config.toml transcripts.csv
    """
    subprocess.run(command, shell=True)

At this stage, you executed 4 times Baysor (once per patch). Now, we need to resolve the conflict, i.e. where boundaries are overlapping due to segmentation on multiple patches:

In [ ]:
from sopa.segmentation.baysor.resolve import resolve

resolve(sdata, baysor_temp_dir, gene_column, min_area=500)

## Aggregate

This **mandatory** step turns the data into an `AnnData` object. We can count the transcript inside each cell, and/or average each channel intensity inside each cell boundary.

> NB: Baysor already counts the transcripts inside each cell to create a cell-by-gene table, so you don't need to provide `gene_column`

In [21]:
aggregator = sopa.segmentation.Aggregator(sdata, image_key=image_key, shapes_key=shapes_key)

aggregator.compute_table(gene_column=gene_column, average_intensities=True)

In [22]:
sdata

Now, `sdata.tables["table"]` is an `AnnData` object.
- If you count the transcripts, then `adata.X` are the raw counts
- If you average the channel intensities, then `adata.X` are the channels intensities
- If you both count the transcript and average the intensities, then `adata.X` are the raw counts, and `adata.obsm["intensities"]` are the channels intensities

## Annotation

#### Option 1: Transcript-based (Tangram)

[Tangram](https://github.com/broadinstitute/Tangram) is a transcript-based annotation that uses an annotated single-cell reference. Let's suppose your reference `AnnData` object is stored in a file called `adata_reference.h5ad` (preferably, keep raw counts), and the cell type is in `adata.obs["cell_type"]`. Then, you can annotate your spatial data as follows:

In [6]:
from sopa.annotation.tangram import tangram_annotate
import anndata

In [ ]:
adata_reference = anndata.read_h5ad("adata_reference.h5ad")

tangram_annotate(sdata, adata_reference, "cell_type")

#### Option 2: Staining-based
For now, our fluorescence-based annotation is very simple. We provide a dictionary where a channel is associated with a population. Then, each cell is associated with the cell type whose corresponding channel is the brightest (according to a certain Z-score). In this tutorial example, we can annotate Tumoral cells, T cells, and B cells:

In [23]:
from sopa.annotation import higher_z_score

marker_cell_dict = {
    "CK": "Tumoral cell",
    "CD20": "B cell",
    "CD3": "T cell"
}

higher_z_score(sdata.tables["table"], marker_cell_dict)

## Pipeline report
You can optionally create an HTML report of the pipeline run (in the example below, we save it under `report.html`). It contains some quality controls for your data.

In [24]:
sopa.io.write_report("report.html", sdata)

## Visualization

### With the Xenium Explorer

The Xenium Explorer is a software developed by 10X Genomics for visualizing spatial data, and it can be downloaded freely [here](https://www.10xgenomics.com/support/software/xenium-explorer/latest). Sopa allows the conversion to the Xenium Explorer, whatever the type of spatial data you worked on. It will create some files under a new `tuto.explorer` directory:

In [25]:
sopa.io.write("tuto.explorer", sdata, image_key, points_key=points_key, gene_column=gene_column)

If you have downloaded the Xenium Explorer, you can now open the results in the explorer: `open tuto.explorer/experiment.xenium` (if using a Unix operating system), or double-click on the latter file.

### With `spatialdata-plot`
[`spatialdata-plot`](https://github.com/scverse/spatialdata-plot) library is a static plotting library for `SpatialData` objects

In [29]:
import spatialdata_plot

In [30]:
sdata\
    .pl.render_points(size=0.01, color="r")\
    .pl.render_images()\
    .pl.render_shapes(shapes_key, outline=True, fill_alpha=0, outline_color="w")\
    .pl.show("global")

## Save your `SpatialData` object

You can save your `SpatialData` object for later use. This will create a `.zarr` directory.

In [ ]:
sdata.write("tuto.zarr")

You can then open the data with `spatialdata`:

In [ ]:
import spatialdata

spatialdata.read_zarr("tuto.zarr")

## Further analyses

- You can read [this tutorial](../spatial) on spatial statistic and geometric analysis.
- You can use [Squidpy](https://squidpy.readthedocs.io/en/latest/index.html) which operates on both the `SpatialData` object or the `AnnData` object, or use other tools of the `scverse` ecosystem such as [`Scanpy`](https://scanpy.readthedocs.io/en/stable/index.html).
- You can also try the CLI or the Snakemake pipeline of Sopa.